# PDF fit

Old version without `K_q` as a free parameter. Will have to use pre v0.1.0 version of PartonDensity to run this. See: https://github.com/cescalara/PartonDensity.jl/pull/68

In [ ]:
using BAT, DensityInterface
using PartonDensity
using QCDNUM
using Plots, Random, Distributions, ValueShapes, ParallelProcessingTools
using StatsBase, LinearAlgebra
using SpecialFunctions, Printf
const sf = SpecialFunctions;
using DelimitedFiles
using LaTeXStrings
using HypothesisTests
using Statistics

gr(fmt=:png);

In [ ]:
import HDF5
samples = bat_read("data/Data-Dirichlet-sysrun-priors-tight.h5").result
#samples = bat_read("data/Data-Dirichlet-sysrun-priors-long.h5").result

open("data/report-sysrun-priors-tight.txt", "w") do io
#open("report-sysrun-priors-long.txt", "w") do io
   show(io, MIME"text/plain"(), bat_report(samples))
end

In [ ]:
qcdnum_grid = QCDNUMGrid(x_min=[1.0e-3, 1.0e-1, 5.0e-1], x_weights=[1, 2, 2], nx=100,
                         qq_bounds=[1.0e2, 3.0e4], qq_weights=[1.0, 1.0], nq=50, spline_interp=3)
qcdnum_params = QCDNUMParameters(order=2, α_S=0.118, q0=100.0, grid=qcdnum_grid,
                                 n_fixed_flav=5, iqc=1, iqb=1, iqt=1, weight_type=1);

splint_params = SPLINTParameters();
quark_coeffs = QuarkCoefficients();

#forward_model_init(qcdnum_grid, qcdnum_params, splint_params)
forward_model_init_sysErr(qcdnum_grid, qcdnum_params, splint_params);

In [ ]:
#
# The following used just to define arrays
#
pdf_params_gen, sim_data = pd_read_sim("data/simulation-Dirichlet-Syst.h5")
nbins=sim_data["nbins"]
counts_obs_ep_sim= sim_data["counts_obs_ep"]
counts_obs_em_sim= sim_data["counts_obs_em"]

#
# Read in the ZEUS data
#
counts_obs_ep_data = readdlm("data/eP.dat") 
counts_obs_em_data = readdlm("data/eM.dat");

In [ ]:
#
# The prior used in the fitting - should read this from a file
#
#
# The tight prior definition on K_u, K_d
#
prior = NamedTupleDist(
    θ = Dirichlet([34.,17.,22.,22.,2,2,1,1,0.5]),
    K_u = Truncated(Normal(3.5, 0.5), 2., 5.),
    K_d = Truncated(Normal(3.5, 0.5), 2., 5.),
    λ_g1 = Uniform(1., 2.),
    λ_g2 = Uniform(-0.5, -0.1),
    K_g =  Truncated(Normal(4., 1.5), 2., 5.),
    λ_q = Uniform(-0.5, -0.1),
    Beta1 =  Truncated(Normal(0, 1), -5, 5),
    Beta2 =  Truncated(Normal(0, 1), -5, 5),
    beta0_1=  Truncated(Normal(0, 1), -5, 5), 
    beta0_2=   Truncated(Normal(0, 1), -5, 5),    
    beta0_3= Truncated(Normal(0, 1), -5, 5), 
    beta0_4=  Truncated(Normal(0, 1), -5, 5), 
    beta0_5=  Truncated(Normal(0, 1), -5, 5), 
    beta0_6=  Truncated(Normal(0, 1), -5, 5), 
    beta0_7=  Truncated(Normal(0, 1), -5, 5), 
    beta0_8=   Truncated(Normal(0, 1), -5, 5)
);

# The loose prior definition on K_u, K_d
#=
       prior = NamedTupleDist(
           θ = Dirichlet([34.,17.,22.,22.,2,2,1,1,0.5]),
           K_u = Truncated(Normal(3.5, 1.), 2., 5.),
           K_d = Truncated(Normal(3.5, 1.), 2., 5.),
           λ_g1 = Uniform(1., 2.),
           λ_g2 = Uniform(-0.5, -0.1),
           K_g =  Truncated(Normal(4., 1.5), 2., 5.),
           λ_q = Uniform(-0.5, -0.1),
           Beta1 =  Truncated(Normal(0, 1), -5, 5),
           Beta2 =  Truncated(Normal(0, 1), -5, 5),
           beta0_1=  Truncated(Normal(0, 1), -5, 5), 
           beta0_2=   Truncated(Normal(0, 1), -5, 5),    
           beta0_3= Truncated(Normal(0, 1), -5, 5), 
           beta0_4=  Truncated(Normal(0, 1), -5, 5), 
           beta0_5=  Truncated(Normal(0, 1), -5, 5), 
           beta0_6=  Truncated(Normal(0, 1), -5, 5), 
           beta0_7=  Truncated(Normal(0, 1), -5, 5), 
           beta0_8=   Truncated(Normal(0, 1), -5, 5)
       );
=#
# Generate samples according to prior for plotting
priorsamples = bat_sample(prior).result

In [ ]:
#
# Here we plot a comparison of the prior and posterior contours in the space of K_
plot(
   [NaN], legend = :topleft, linecolor = :blue4,
   label = L"P_0(K_u,\Delta_u), 95~\%")
plot!(priorsamples, (:(K_u), :(θ[1])),xrange=(2.,5),yrange=(0,1),seriestype=:smallest_intervals_contour,linecolor=:blue4,    
intervals = [0.955],smoothing=4,marginalmode = false,
    legend=:topleft,
    label=L"P_0(K_u,\Delta_u), 95~\%",)
plot!(
   [NaN], legend = :topleft, linecolor = :blue,
   label = L"P_0(K_u,\Delta_u), 68~\%")
plot!(priorsamples, (:(K_u), :(θ[1])),xrange=(2.,5),yrange=(0,1),seriestype=:smallest_intervals_contour,linecolor=:blue,    
intervals = [0.68], smoothing=4,marginalmode = false,
    legend=:topleft,
    label=L"P_0(K_u,\Delta_u), 68~\%",)
plot!(
    samples, (:(K_u), :(θ[1])),xrange=(2.,5),yrange=(0,1.),
    intervals = [0.955],
    nbins=100,
    colors=[:green],
    smoothing=4,
    alpha=0.7,
    marginalmode=false,
    legend=:topleft,
    label=L"P(K_u,\Delta_u), 95~\%",
    
)
plot!(
    samples, (:(K_u), :(θ[1])),xrange=(2.,5),yrange=(0,1.),
    intervals = [0.683],
    nbins=100,
    colors=[:green2],
    smoothing=4,
    alpha=0.7,
    marginalmode=false,
    legend=:topleft,
    label=L"P(K_u,\Delta_u), 68~\%",
    xlabel=L"K_u",
    ylabel=L"\Delta_u",
#    legendfontsize=3,
    thickness_scaling=1.3
    
)

In [ ]:
savefig("Mom-K.pdf")

In [ ]:
#
# Comparison of prior and posterior for momentum of u-valence
#
p1=plot(priorsamples, vsel=[:(θ[1])],xrange=(0,1),seriestype=:stephist,
        label=L"P_0(\Delta_u)",
)
#=
p1=plot!(samples, vsel=[:(θ[1])],xrange=(0,1),    
    xlabel=L"\Delta_u",
    ylabel=L"P(\Delta_u)",
    label=L"68~\%~{\rm Credible \; Interval",
    thickness_scaling=1.1,
    legend=:topright,
        alpha=0.5,
    intervals = [0.68],
    )
=#
p1=plot!(samples, vsel=[:(θ[1])],xrange=(0,1),    
    xlabel=L"\Delta_u",
    ylabel=L"P_0(\Delta_u), P(\Delta_u)",
    label=L"68~\%~{\rm Credible \; Interval",
    thickness_scaling=1.3,
    legend=:none,
        alpha=0.5,
    intervals = [0.68],
    )
plot(p1)

In [ ]:
#
# Comparison of prior and posterior for K_u
#
p2=plot(priorsamples, vsel=[:K_u],xrange=(2,5),seriestype=:stephist,
            label=L"P_0(K_u)",
    )
p2=plot!(samples, vsel=[:K_u],xrange=(2,5),    
    xlabel=L"K_u",
    ylabel=L"P_0(K_u), P(K_u)",
    label=L"68~\%~{\rm Credible \; Interval",
    thickness_scaling=1.3,
    legend=:topright,
    alpha=0.5,
    intervals = [0.68],
    )
#plot(p2)

In [ ]:
#
# Stacked plots
#
plot(p1,p2, layout = (2, 1))

In [ ]:
savefig("Marginals-momu-Ku.pdf")

In [ ]:
#
# Here we calculated the effective exponent of (1-x) for the u-valence distribution
#
x_range=0.5:0.01:1
f(v,x)=v.K_u-(1-x)*v.θ[1]*(v.K_u+1)/(2-v.θ[1])/x
plot(
   x_range, f, samples, yrange=(2,5),
   intervals = [0.68],
#   colors = [:green4],
   global_mode = false, median = false,
   label = L"68~\%~{\rm Credible \; Interval}",
    xlabel=L"x",
    ylabel=L"\beta(x)",
    legend=:none

)

In [ ]:
savefig("EffExp.pdf")

In [ ]:
#
# And here the u-valence distribution itself
#
function wrap_xuVal(p::NamedTuple, x::Real)
    
    pdf_params3 = DirichletPDFParams(K_u=p.K_u, K_d=p.K_d, λ_g1=p.λ_g1, 
                                    λ_g2=p.λ_g2, K_g=p.K_g, λ_q=p.λ_q, θ=p.θ)
    uval = PartonDensity.x_uv_x(x,pdf_params3.λ_u,pdf_params3.K_u  )
    return uval
end
x_grid = range(0, stop=1, length=200)
plot(x_grid, wrap_xuVal, samples, intervals=[0.68],colors=[:chartreuse2],
   label = L"68~\%~{\rm Credible \; Interval}",
    xlabel=L"x",
    ylabel=L"xu(x,Q^2=100~{\rm GeV}^2)",
       global_mode = false, median = false,
    legend=:topright
)

In [ ]:
savefig("xuvalence.pdf")

In [ ]:
#
# Here we calculated the effective exponent of (1-x) for the u-valence distribution
#
f(v)=v.θ[1]*(v.K_u+1)/(2-v.θ[1])


In [ ]:
mode_pars_data = mode(samples)
f(mode_pars_data)

In [ ]:
sub_samples = BAT.bat_sample(samples, BAT.OrderedResampling(nsamples=10000)).result

lambda=zeros(10000)
for i=1:10000
    pdf_params_i = DirichletPDFParams(K_u=sub_samples.v.K_u[i], K_d=sub_samples.v.K_d[i],
                                          λ_g1=sub_samples.v.λ_g1[i], λ_g2=sub_samples.v.λ_g2[i],
                                          K_g=sub_samples.v.K_g[i], λ_q=sub_samples.v.λ_q[i], 
                                          θ=Vector(sub_samples.v.θ[i]))
    lambda[i]=f(pdf_params_i)
end
println(quantile!(lambda, 0.5)," ",quantile!(lambda, 0.16)," ",quantile!(lambda, 0.84))
